### repo breakdown
- something wrong with get_acts()


- steps: 
    - get activations (debugged)
    - train a classifier (debugged)
    - load dictionaries (ok)
    - build feature circuits graph (Debugging)
    - edit feature circuits graph (TBC): evaluate its impact on generative responses?

    - train dictionaries (TBC)

- exp setup: 
    - harmful/harmless dataset

note: 
- tokenisers return attention masks..
- hidden states are tuples? (could return tuples?)
- batch encoding of inputs (takes the longest input sequence)
    - eg. (8, 126): batch of 8 inputs, longest input is 126 tokens 
    - hence activation dimensions: 8, 126, 512

In [ ]:
### things i want to learn more about: under the hood
## ml-ops
## python context object (with), defines logic for entering and exiting this context
## eg. open file, read file, close file; open model, read model, close model

## 1. build concept classifier (rep for the concept)
gather feature representations for classifying concept
> (biographies of male/female prof and nurse) -> profession: 
- to identify the gender features relevant for classification


> (harmful prompt, harmless prompt) -> 0, 1: 
- to identify most relevant features 



In [1]:
import torch as t

In [2]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from datasets import load_dataset


/common/home/users/x/xinrose.lin.2020/.conda/envs/fcs-exp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from nnsight import LanguageModel

#  model hyperparameters
# DEVICE = 'cuda:0'
DEVICE = "cpu"
model = LanguageModel('EleutherAI/pythia-70m-deduped', device_map=DEVICE, dispatch=True)
activation_dim = 512

In [2]:

inputs = model.tokenizer('the eiffel tower is')
inputs

{'input_ids': [783, 299, 1648, 293, 15469, 310], 'attention_mask': [1, 1, 1, 1, 1, 1]}

#### process dataset

In [19]:
# dataset hyperparameters
dataset = load_dataset("LabHC/bias_in_bios")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 257478
    })
    test: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 99069
    })
    dev: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 39642
    })
})

In [20]:
dataset['train'][:5]

{'hard_text': ['He is also the project lead of and major contributor to the open source assembler/simulator "EASy68K." He earned a master’s degree in computer science from the University of Michigan-Dearborn, where he is also an adjunct instructor. Downloads/Updates',
  'She is able to assess, diagnose and treat minor illness conditions and exacerbations of some long term conditions. Her qualifications include Registered General Nurse, Bachelor of Nursing, Diploma in Health Science, Emergency Care Practitioner and Independent Nurse Prescribing.',
  'Prior to law school, Brittni graduated magna cum laude from DePaul University in 2011 with her Bachelor’s Degree in Psychology and Spanish. In 2014, she earned her law degree from Chicago-Kent College of Law. While at Chicago-Kent, Brittni was awarded two CALI Excellence for the Future Awards in both Legal Writing and for her seminar article regarding President Obama’s executive action, Deferred Action for Childhood Arrivals.',
  'He regula

In [4]:
import torch as t
import random

# dataset hyperparameters
dataset = load_dataset("LabHC/bias_in_bios")
profession_dict = {'professor' : 21, 'nurse' : 13}
male_prof = 'professor'
female_prof = 'nurse'

# data preparation hyperparameters
batch_size = 8
SEED = 42

# To fit on 24GB VRAM GPU, I set the next 2 default batch_sizes to 64
def get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED):
    if train:
        data = dataset['train']
    else:
        data = dataset['test']
    if ambiguous:
        #
        neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        n = min([len(neg), len(pos)])
        neg, pos = neg[:n], pos[:n]
        data = neg + pos
        labels = [0]*n + [1]*n
        idxs = list(range(2*n))
        random.Random(seed).shuffle(idxs)
        data, labels = [data[i] for i in idxs], [labels[i] for i in idxs]
        true_labels = spurious_labels = labels
    else:
        ## this dataset is used; extract from dataset if:
        ## neg neg: male profession, male gender (0)
        ## neg pos: male profession, female gender (1)
        ## pos neg: female profession, male gender (0)
        ## pos pos: female profession, female gender (1)
        neg_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        neg_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 1]
        pos_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 0]
        pos_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        ## to extract an equal split for all categories
        n = min([len(neg_neg), len(neg_pos), len(pos_neg), len(pos_pos)])
        neg_neg, neg_pos, pos_neg, pos_pos = neg_neg[:n], neg_pos[:n], pos_neg[:n], pos_pos[:n]
        data = neg_neg + neg_pos + pos_neg + pos_pos
        true_labels     = [0]*n + [0]*n + [1]*n + [1]*n
        spurious_labels = [0]*n + [1]*n + [0]*n + [1]*n
        idxs = list(range(4*n))
        random.Random(seed).shuffle(idxs)
        data, true_labels, spurious_labels = [data[i] for i in idxs], [true_labels[i] for i in idxs], [spurious_labels[i] for i in idxs]

    batches = [
        (data[i:i+batch_size], t.tensor(true_labels[i:i+batch_size], device=DEVICE), t.tensor(spurious_labels[i:i+batch_size], device=DEVICE)) for i in range(0, len(data), batch_size)
    ]

    return batches

# (text, profession, gender)

In [22]:
batches = get_data(train=False, ambiguous=False)
batches[:5]

[(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
   'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart a

In [26]:
print('batches: ', len(batches))
print('each batch: ', len(batches[0][0]))
print('batch sample: ', batches[0][1])
print('batch dtype: ', type(batches[0]), type(batches[0][0]))
## (text, profession, gender)

batches:  217
each batch:  8
batch sample:  LanguageModelProxy (tensor_0): FakeTensor(..., size=(8,), dtype=torch.int64)
batch dtype:  <class 'tuple'> <class 'list'>


In [29]:
type(batches[0][0][0])

str

In [ ]:
def get_subgroups(train=True, ambiguous=True, batch_size=128, seed=SEED):
    if train:
        data = dataset['train']
    else:
        data = dataset['test']
    if ambiguous:
        neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        neg_labels, pos_labels = (0, 0), (1, 1)
        subgroups = [(neg, neg_labels), (pos, pos_labels)]
    else:
        neg_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        neg_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 1]
        pos_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 0]
        pos_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        neg_neg_labels, neg_pos_labels, pos_neg_labels, pos_pos_labels = (0, 0), (0, 1), (1, 0), (1, 1)
        
        subgroups = [(neg_neg, neg_neg_labels), (neg_pos, neg_pos_labels), (pos_neg, pos_neg_labels), (pos_pos, pos_pos_labels)]
    
    out = {}
    for data, label_profile in subgroups:
        out[label_profile] = []
        for i in range(0, len(data), batch_size):
            text = data[i:i+batch_size]
            out[label_profile].append(
                (
                    text,
                    t.tensor([label_profile[0]]*len(text), device=DEVICE),
                    t.tensor([label_profile[1]]*len(text), device=DEVICE)
                )
            )
    return out

#### classifier
- linear
- ARE: classifier is nonlinear?

In [4]:
from torch import nn

# probe training hyperparameters

layer = 4 # model layer for attaching linear classification head

class Probe(nn.Module):
    def __init__(self, activation_dim):
        super().__init__()
        self.net = nn.Linear(activation_dim, 1, bias=True)

    def forward(self, x):
        logits = self.net(x).squeeze(-1)
        return logits

### from ARE repo
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    # @autocast()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        #x = nn.functional.softmax(x, dim=1)
        return x

In [3]:
probe = Probe(512) ## Nonetype error, could be an import issue?

#### aside: nnsight trace context

In [10]:
## python context object (with), defines logic for entering and exiting this context
## eg. open file, read file, close file; open model, read model, close model

## model only executed at the end of tracing context 
## .input .output within context requires .save() to be accessed at the end of context

from nnsight import NNsight
from collections import OrderedDict

## ordered-dict: for accessing via keys
net = nn.Sequential(
    OrderedDict(
        [
            ('layer1', nn.Linear(5, 10)), 
            ('layer2', nn.Linear(10, 2))
    ]  
    )
).requires_grad_(False)

tiny_model = NNsight(net)

input = t.rand((1, 5))

with tiny_model.trace(input) as tracer:
    input = tiny_model.input.save()
    ## input + attention mask if there is: (from tokenizer?)
    ## usually specific to model
    tracer.log('model inputs', tiny_model.inputs)
    layer1 = tiny_model.layer1.output.save()
    ## can also log to save memory; if we dont need it after
    tracer.log('model out', tiny_model.output)

print(input.value)
print(layer1.value)

model inputs ((tensor([[0.0486, 0.0228, 0.9779, 0.3312, 0.3188]]),), {})
model out tensor([[0.2775, 0.3287]])
tensor([[0.0486, 0.0228, 0.9779, 0.3312, 0.3188]])
tensor([[ 0.2449, -0.6229,  0.5052,  0.5031,  0.4278, -0.3936,  0.4124, -0.2411,
          0.3344, -0.5219]])


In [80]:
input.shape

torch.Size([1, 5])

In [72]:
print('tiny model', type(tiny_model))
print('layers', type(tiny_model.layer2)) ### envoy??

tiny model <class 'nnsight.models.NNsightModel.NNsight'>
layers <class 'nnsight.envoy.Envoy'>


In [67]:
## function, methods, operations can happen within context 
## will simply be executed when model is ran
## only values with .save() will be saved
## HOWEVER, since tracing context operates on intervention graph
## to apply a custom function, it needs to be part of the intervention graph

with tiny_model.trace(input):
    value = (tiny_model.layer1.output.sum() + tiny_model.layer2.output.sum()).save()
print(value)

tensor(-2.3806)


In [34]:
sample_text = batches[0][0][0]

In [85]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [32]:
model.gpt_neox.layers[4]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXSdpaAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_dropout): Dropout(p=0.0, inplace=False)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
    (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
    (act): GELUActivation()
  )
)

In [1]:
from nnsight import LanguageModel

llm = LanguageModel("openai-community/gpt2", device_map="auto")

print(llm)

/common/home/users/x/xinrose.lin.2020/.conda/envs/fcs-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  (generator): WrapperModule()
)


In [2]:
## why did it receive lang model proxy??

# .trace() creates two diff context, model and invoker
# invoker: defines values of input and output Proxies
# .trace(...) inputs are passed to invoker

with llm.trace("The Eiffel Tower is in the city of") as tracer:
    tracer.log(llm.transformer.input.shape)
    # Access the last layer using h[-1] as it's a ModuleList
    # Access the first index of .output as that's where the hidden states are.
    tracer.log(llm.transformer.h[-1].mlp.output.shape)
    tracer.log(llm.transformer.h[-1].mlp.output[0][:].shape)
    llm.transformer.h[-1].mlp.output[0][:] = 0

    # Logits come out of model.lm_head and we apply argmax to get the predicted token ids.
    token_ids = llm.lm_head.output.argmax(dim=-1).save()

print("\nToken IDs:", token_ids)

# Apply the tokenizer to decode the ids into words after the tracing context.
print("Prediction:", llm.tokenizer.decode(token_ids[0][-1]))

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([1, 10])
torch.Size([1, 10, 768])
torch.Size([10, 768])

Token IDs: tensor([[ 262,   12,  417, 8765,   11,  257,  262, 3504,  338, 3576]])
Prediction:  London


##### get activations as a batch

In [30]:
# batching 

with llm.trace() as tracer:

    with tracer.invoke("The Eiffel Tower is in the city of"):
        
        # Ablate the last MLP for only this batch.
        llm.transformer.h[-1].mlp.output[0][:] = 0

        # Get the output for only the intervened on batch.
        token_ids_intervention = llm.lm_head.output.argmax(dim=-1).save()

    with tracer.invoke("The Eiffel Tower is in the city of"):

        # Get the output for only the original batch.
        token_ids_original = llm.lm_head.output.argmax(dim=-1).save()

print("Original token IDs:", token_ids_original)
print("Modified token IDs:", token_ids_intervention)

print("Original prediction:", llm.tokenizer.decode(token_ids_original[0][-1]))
print("Modified prediction:", llm.tokenizer.decode(token_ids_intervention[0][-1]))

Original token IDs: tensor([[ 198,   12,  417, 8765,  318,  257,  262, 3504, 7372, 6342]])
Modified token IDs: tensor([[ 262,   12,  417, 8765,   11,  257,  262, 3504,  338, 3576]])
Original prediction:  Paris
Modified prediction:  London


In [20]:
## graph error = graph node access error? graph node not found?

with llm.trace(["The Eiffel Tower is in the city of", "The Eiffel Tower is in the city of"]) as tracer:
    tracer.log(llm.transformer.input)
    # Access the last layer using h[-1] as it's a ModuleList
    # Access the first index of .output as that's where the hidden states are.
    tracer.log(llm.transformer.h[-1].mlp.output.shape)
    tracer.log(llm.transformer.h[-1].mlp.output[0][:].shape)
    llm.transformer.h[-1].mlp.output[0][:] = 0

    # Logits come out of model.lm_head and we apply argmax to get the predicted token ids.
    token_ids = llm.lm_head.output.argmax(dim=-1).save()

print("\nToken IDs:", token_ids)

# Apply the tokenizer to decode the ids into words after the tracing context.
print("Prediction:", llm.tokenizer.decode(token_ids[0][-1]))

tensor([[ 464,  412,  733,  417, 8765,  318,  287,  262, 1748,  286],
        [ 464,  412,  733,  417, 8765,  318,  287,  262, 1748,  286]])
torch.Size([2, 10, 768])
torch.Size([10, 768])

Token IDs: tensor([[ 262,   12,  417, 8765,   11,  257,  262, 3504,  338, 3576],
        [ 198,   12,  417, 8765,  318,  257,  262, 3504, 7372, 6342]])
Prediction:  London


In [ ]:
llm

In [19]:
tokens = llm.tokenizer.encode("The Eiffel Tower is in the city of")
print("tokenized:", llm.tokenizer.decode(tokens))
print(tokens)
len(tokens)

tokenized: The Eiffel Tower is in the city of
[464, 412, 733, 417, 8765, 318, 287, 262, 1748, 286]


10

In [6]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [34]:
with model.trace(["The Eiffel Tower is in the city of","The Eiffel Tower is in the city of"]) as tracer:
    tracer.log(model.inputs)
    tracer.log(model.gpt_neox.input) 
    tracer.log(model.gpt_neox.layers[-1].mlp.output.shape)

    # Access the last layer using h[-1] as it's a ModuleList
    # Access the first index of .output as that's where the hidden states are.
    # model.layers[-1].output[0][:] = 0

    # Logits come out of model.lm_head and we apply argmax to get the predicted token ids.
    tracer.log(model.embed_out.output.shape)
    token_ids = model.embed_out.output.argmax(dim=-1).save()

print("\nToken IDs:", token_ids)

# Apply the tokenizer to decode the ids into words after the tracing context.
print("Prediction:", llm.tokenizer.decode(token_ids[0][-1]))

((), {'input_ids': tensor([[  510,   444,  1648,   293, 20610,   310,   275,   253,  2846,   273],
        [  510,   444,  1648,   293, 20610,   310,   275,   253,  2846,   273]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])})
tensor([[  510,   444,  1648,   293, 20610,   310,   275,   253,  2846,   273],
        [  510,   444,  1648,   293, 20610,   310,   275,   253,  2846,   273]])
torch.Size([2, 10, 512])
torch.Size([2, 10, 50304])

Token IDs: tensor([[1533,   15,  293,   13,  187,  247,  253, 2798,  273,  253],
        [1533,   15,  293,   13,  187,  247,  253, 2798,  273,  253]])
Prediction: �


In [41]:
tokens = model.tokenizer.encode("The Eiffel Tower is in the city of")
print(len(tokens))
tokens = model.tokenizer.encode("The london bridge is in the city of")
print(len(tokens))

10
9


#### load data

In [94]:
batches[0]

(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
  'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and

In [99]:
import numpy as np 

new_batch = []
for batch in batches: 
    prof = batch[1].tolist()
    gender = batch[2].tolist()
    new_batch.append((batch[0], prof, gender))
    

In [101]:
new_batch[0]

(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
  'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and

In [102]:
import json
# File path to save the JSON file
file_path = 'exp_redo/batched_data.json'

# Writing list to JSON file
with open(file_path, 'w') as json_file:
    json.dump(new_batch, json_file)

print(f"List saved to {file_path}")

List saved to exp_redo/batched_data.json


#### load processed json (balanced dataset)

In [12]:
import json
import torch as t 

# Specify the file path of the JSON file
file_path = 'exp_redo/batched_data.json'

# Open and load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)
    batches = []
    for batch in data: 
        prof = t.tensor(batch[1])
        gender = t.tensor(batch[2])
        batches.append((batch[0], prof, gender))

# Print or work with the loaded data
print(batches)

[(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.", 'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and 

In [12]:
batches[:5]

[(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
   'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart a

#### get activations / feature representations

In [31]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [36]:
## get activations across a batch of text input
text = batches[0][0]
layer = 4
# tracer_kwargs = dict(scan=True, validate=True)
with t.no_grad(): 
    with model.trace(text) as tracer:
        # tracer.log(model.gpt_neox.embed_in)
        tracer.log('in')
        ## TODO: input does not have attention mask.. 
        ## model.gpt_neox.input is literally referring to the input
        ## MODEL.INPUTS rather is 
        attn_mask = model.inputs[1]['attention_mask']
        # tracer.log(attn_mask)
        print(attn_mask.output.shape)
        acts = model.gpt_neox.layers[layer].output[0]
        acts = acts * attn_mask[:, :, None]
        acts = acts.sum(1) / attn_mask.sum(1)[:, None]
        acts = acts.save()
    # return acts.value

in


In [27]:
DEBUGGING = False

if DEBUGGING:
    tracer_kwargs = dict(scan=True, validate=True)
else:
    tracer_kwargs = dict(scan=False, validate=False)
    

def get_acts(text):
    with t.no_grad(): 
        with model.trace(text, **tracer_kwargs) as tracer:
            ## TODO: note attention mask..?
            tracer.log(model.gpt_neox.input)
            attn_mask = model.input[1]['attention_mask']
            acts = model.gpt_neox.layers[layer].output[0]
            acts = acts * attn_mask[:, :, None]
            acts = acts.sum(1) / attn_mask.sum(1)[:, None]
            acts = acts.save()
        return acts.value

In [30]:
batches=get_data()
batches[:5]

[(['She remains actively involved with the Burn Treatment Center after working there for nearly 18 years. Throughout these years she was a Nursing Assistant, Staff Nurse, Assistant Nurse Manager, and most recently the Nurse Manager. She is a member of the American Burn Association, the American Association of Critical Care Nurses, and the Health Information Management Systems Society. She has been co-director of Miracle Burn Camp since 2001 & assisted in creating the Young Adult Burn Survivor retreat in 2007. Alison is the mother of two and stays active as a board member for the local youth hockey association and keeping up with her boys. To Contact Alison, please: *protected email*',
   'She graduated with honors in 1999. Having more than 17 years of diverse experiences, especially in NURSE PRACTITIONER, Shannon R Russom affiliates with University Of Vermont Medical Center, and cooperates with other doctors and specialists in medical group University Of Vermont Medical Center Inc. Cal

In [37]:
def train_probe(get_acts, label_idx=0, batches=get_data(), lr=1e-2, epochs=1, dim=512, seed=SEED):
    t.manual_seed(seed)
    probe = Probe(dim).to(DEVICE)
    optimizer = t.optim.AdamW(probe.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    losses = []
    for epoch in range(epochs):
        for batch in batches:
            
            text = batch[0]
            labels = batch[label_idx+1] 
            acts = get_acts(text)
            logits = probe(acts)
            loss = criterion(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

    return probe, losses

In [ ]:
def test_probe(probe, get_acts, label_idx=0, batches=get_data(train=False), seed=SEED):
    with t.no_grad():
        corrects = []

        for batch in batches:
            text = batch[0]
            labels = batch[label_idx+1]
            acts = get_acts(text)
            logits = probe(acts)
            preds = (logits > 0.0).long()
            corrects.append((preds == labels).float())
        return t.cat(corrects).mean().item()

#### probe training

In [ ]:
# probe training hyperparameters

layer = 4 # model layer for attaching linear classification head

class Probe(nn.Module):
    def __init__(self, activation_dim):
        super().__init__()
        self.net = nn.Linear(activation_dim, 1, bias=True)

    def forward(self, x):
        logits = self.net(x).squeeze(-1)
        return logits

def train_probe(get_acts, label_idx=0, batches=get_data(), lr=1e-2, epochs=1, dim=512, seed=SEED):
    t.manual_seed(seed)
    probe = Probe(dim).to(DEVICE)
    optimizer = t.optim.AdamW(probe.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    losses = []
    for epoch in range(epochs):
        for batch in batches:
            text = batch[0]
            labels = batch[label_idx+1] 
            acts = get_acts(text)
            logits = probe(acts)
            loss = criterion(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

    return probe, losses

def test_probe(probe, get_acts, label_idx=0, batches=get_data(train=False), seed=SEED):
    with t.no_grad():
        corrects = []

        for batch in batches:
            text = batch[0]
            labels = batch[label_idx+1]
            acts = get_acts(text)
            logits = probe(acts)
            preds = (logits > 0.0).long()
            corrects.append((preds == labels).float())
        return t.cat(corrects).mean().item()
    
def get_acts(text):
    with t.no_grad(): 
        with model.trace(text, **tracer_kwargs):
            attn_mask = model.input[1]['attention_mask']
            acts = model.gpt_neox.layers[layer].output[0]
            acts = acts * attn_mask[:, :, None]
            acts = acts.sum(1) / attn_mask.sum(1)[:, None]
            acts = acts.save()
        return acts.value

In [ ]:
batches = get_data(train=False, ambiguous=False)
print("ground truth accuracy:", test_probe(oracle, get_acts, batches=batches, label_idx=0))
print("unintended feature accuracy:", test_probe(oracle, get_acts, batches=batches, label_idx=1))

In [ ]:
### in the end the error was model.inputs ... (syntax)

## 2. SAE latents of model
- load dictionaries of model: 
    - embed + 5 (0, 1, 2, 3, 4 layers) * 3
    - 16 dictionaries / submodules 
    - Autoencoders: 512 * 32768
- train dictionaries of model


In [19]:
print(type(model.gpt_neox.embed_in))
# Envoy object act as proxies for torch modules within a model’s module 
# tree in order to add nnsight functionality. Proxies of the underlying 
# module’s output and input are accessed by .output and .input respectively.
model.gpt_neox.embed_in

<class 'nnsight.envoy.Envoy'>


Embedding(50304, 512)

#### load SAE

In [5]:
# loading dictionaries
from dictionary_learning import AutoEncoder

# dictionary hyperparameters
dict_id = 10
expansion_factor = 64
dictionary_size = expansion_factor * activation_dim
DEVICE = 'cpu'
submodules = []
dictionaries = {}
path = "dictionary_learning"
layer = 4 

submodules.append(model.gpt_neox.embed_in)
dictionaries[model.gpt_neox.embed_in] = AutoEncoder.from_pretrained(
    f'{path}/dictionaries/pythia-70m-deduped/embed/{dict_id}_{dictionary_size}/ae.pt',
    device=DEVICE
)
for i in range(layer + 1):
    submodules.append(model.gpt_neox.layers[i].attention)
    dictionaries[model.gpt_neox.layers[i].attention] = AutoEncoder.from_pretrained(
        f'{path}/dictionaries/pythia-70m-deduped/attn_out_layer{i}/{dict_id}_{dictionary_size}/ae.pt',
        device=DEVICE
    )

    submodules.append(model.gpt_neox.layers[i].mlp)
    dictionaries[model.gpt_neox.layers[i].mlp] = AutoEncoder.from_pretrained(
        f'{path}/dictionaries/pythia-70m-deduped/mlp_out_layer{i}/{dict_id}_{dictionary_size}/ae.pt',
        device=DEVICE
    )

    submodules.append(model.gpt_neox.layers[i])
    dictionaries[model.gpt_neox.layers[i]] = AutoEncoder.from_pretrained(
        f'{path}/dictionaries/pythia-70m-deduped/resid_out_layer{i}/{dict_id}_{dictionary_size}/ae.pt',
        device=DEVICE
    )

def metric_fn(model, labels=None):
    attn_mask = model.inputs[1]['attention_mask']
    acts = model.gpt_neox.layers[layer].output[0]
    acts = acts * attn_mask[:, :, None]
    acts = acts.sum(1) / attn_mask.sum(1)[:, None]
    
    return t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    )

# somehow middle layer is chosen for training the class
# middle layer - expected to be where higher level features are learnt?

/common/home/users/x/xinrose.lin.2020/feature-circuits/dictionary_learning/dictionary.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t.load(path, map_loca

#### eda on dictionaries

In [20]:
dictionaries[model.gpt_neox.embed_in] # 512 * 64

AutoEncoder(
  (encoder): Linear(in_features=512, out_features=32768, bias=True)
  (decoder): Linear(in_features=32768, out_features=512, bias=False)
)

In [40]:
dictionaries[model.gpt_neox.layers[0].mlp]

AutoEncoder(
  (encoder): Linear(in_features=512, out_features=32768, bias=True)
  (decoder): Linear(in_features=32768, out_features=512, bias=False)
)

In [41]:
dictionaries[model.gpt_neox.layers[0].attention]

AutoEncoder(
  (encoder): Linear(in_features=512, out_features=32768, bias=True)
  (decoder): Linear(in_features=32768, out_features=512, bias=False)
)

In [43]:
dictionaries[model.gpt_neox.layers[0]]

AutoEncoder(
  (encoder): Linear(in_features=512, out_features=32768, bias=True)
  (decoder): Linear(in_features=32768, out_features=512, bias=False)
)

In [32]:
sq = nn.Linear(5, 2) ## (2x5 matrix)
sq

In [26]:
ae_embed_in = dictionaries[model.gpt_neox.embed_in] # 512 * 64
ae_embed_in.encoder.state_dict()['weight'].shape # tensor of 32768, 512

torch.Size([32768, 512])

In [30]:
ae_embed_in.encoder.state_dict()['bias'].shape

torch.Size([32768])

In [29]:
ae_embed_in.encoder.state_dict()['weight'][0].shape

torch.Size([512])

In [27]:
ae_embed_in.decoder.state_dict()['weight'].shape

torch.Size([512, 32768])

#### [TBC] train SAE

## 3. build casual graph for concept classifier
- approximate causal effect of the feature: 
    - atp
    - ig
    - exact

- find the most causally relevant latent features for classification performance

In [64]:
### load data
import json
import torch as t 

# Specify the file path of the JSON file
file_path = 'exp_redo/batched_data.json'

# Open and load the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)
    batches = []
    for batch in data: 
        prof = t.tensor(batch[1])
        gender = t.tensor(batch[2])
        batches.append((batch[0], prof, gender))

# Print or work with the loaded data
print(batches)
print(len(batches))

[(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.", 'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and 

In [65]:
clean = batches[0][0] # clean data: just referring text data, not corrupted/patched
labels = batches[0][1]

In [66]:
labels

tensor([1, 1, 1, 1, 1, 1, 1, 1])

In [20]:
clean

["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
 'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and L

#### load pretrained probe

In [7]:
### load trained concept probe

# Define the model structure
probe = Probe(512)  # Replace with the actual model class

# Load the state dictionary
probe.load_state_dict(t.load('exp_redo/24_10_probe_model_state.pth'))

# (Optional) Set the model to evaluation mode
probe.eval()

/tmp/ipykernel_3038011/3062431710.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  probe.load_state_dict(t.load('exp_redo/24_10_probe_model_state.pth'))


Probe(
  (net): Linear(in_features=512, out_features=1, bias=True)
)

In [10]:
### tokenise

### get hidden state via hooks 
### get attention mask 
model.tokenizer(clean)
### register_forward_hooks to get activations 
### gather activations wrt to attention mask 

{'input_ids': [[2993, 20981, 342, 34727, 432, 2499, 4683, 1457, 8987, 4726, 4683, 13715, 275, 7544, 15, 14566, 625, 685, 1722, 1107, 273, 11117, 8450, 13, 3340, 275, 330, 6366, 3801, 16577, 427, 3322, 2354, 38637, 56, 37272, 13, 31603, 90, 3344, 3248, 36360, 35751, 342, 1142, 13240, 1690, 49668, 19804, 8710, 5197, 13, 21519, 4214, 8896, 13, 285, 10239, 684, 342, 643, 11576, 285, 27371, 275, 3739, 1387, 10168, 434, 8710, 5197, 20288, 81, 15, 9368, 31603, 90, 3344, 3248, 36360, 327, 4481, 1180, 313, 41211, 10, 818, 3763, 14, 1438, 2417, 323, 625, 1491, 285, 1604, 3013, 390, 281, 1984, 271, 14286, 15], [1328, 4390, 2987, 275, 253, 2798, 11286, 285, 2798, 4433, 5659, 15, 754, 556, 2793, 342, 23066, 13, 19170, 13, 285, 6782, 2570, 28296, 2798, 2728, 13, 50098, 13, 285, 11286, 15, 754, 4307, 323, 689, 884, 1107, 275, 253, 44039, 708, 46283, 837, 263, 23220, 25311, 15, 754, 671, 310, 247, 15022, 42985, 496, 26161, 15, 754, 310, 271, 3939, 3558, 273, 253, 2448, 7115, 273, 34448, 14, 17784, 304

In [49]:
batches[0][0]

["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
 'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and L

#### metric function

In [38]:
def metric_fn(model, probe, labels=None):
    # print(model.input)
    attn_mask = model.inputs[1]['attention_mask']
    acts = model.gpt_neox.layers[layer].output[0]
    
    print(model.gpt_neox.layers[layer].output[0])
    acts = acts * attn_mask[:, :, None]
    acts = acts.sum(1) / attn_mask.sum(1)[:, None]
    # print(acts)
    ## if label = 0 (men?), return probe score, given activations
    # ## if label = 1, return
    # print('probe')
    # print(probe(acts))
    # print('metric')
    pred = probe(acts)
    print(pred)
    metric = t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    )
    # return acts
    return metric

In [81]:
for data in clean: 
    tokens = model.tokenizer.encode(data)
    print(len(tokens))
## batch encoding; 
## 

103
126
96
87
55
71
34
93


In [71]:
model.tokenizer(clean)[0]

Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [78]:
# scan=True, validate=True
### NOTE: batch encoding dimensoins: takes input of longest length, pads the rest
### there is attention mask: corresponding to the padding / input length
import nnsight
with model.trace(clean) as tracer, t.no_grad():
    # tracer.log(model.inputs[1]['input_ids'])
    # tracer.log(model.inputs[1]) # (8, 126)
    attn_mask = model.inputs[1]['attention_mask']
    # tracer.log('attn_mask', attn_mask, 'attn_mask_shape: ____', attn_mask.shape)
    acts = model.gpt_neox.layers[layer].output[0]
    # tracer.log(acts, acts.shape)
    # metric_clean = metric_fn(model, labels).save()
# metric_clean.shape

{'input_ids': tensor([[    0,     0,     0,  ...,   271, 14286,    15],
        [ 1328,  4390,  2987,  ..., 15382,  6807,    15],
        [    0,     0,     0,  ...,  2144,  5445,    15],
        ...,
        [    0,     0,     0,  ..., 25238, 11946,    15],
        [    0,     0,     0,  ...,  3928, 37870,    15],
        [    0,     0,     0,  ...,   271, 14286,    15]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])}


#### debug 2: 
- (when debugging, breaking things down, important to note input output type of the original code ><)
- batch input encoding (batch_size, max_input_length)
- attn mask is kinda mainly: to note the padding of inputs (shorter inputs are padded to match length of max input)
- t.where need to be received as a tensor (tensor condition)
seems like output is a tuple? hence...

- model.output : is a tuple (output, dynamic cache??)
    - ought to be model.output[0]



In [63]:
with model.trace(clean) as tracer, t.no_grad():
        attn_mask = model.inputs[1]['attention_mask'] # (batch, length): (8, 126)
        acts = model.gpt_neox.layers[layer].output
        tracer.log(acts[1])
        ## NOTE

DynamicCache()


In [61]:
with model.trace(clean) as tracer, t.no_grad():
        attn_mask = model.inputs[1]['attention_mask'] # (batch, length): (8, 126)
        acts = model.gpt_neox.layers[layer].output[0] # (batch, length, dim): ((8, 126, 512)
        tracer.log('output :', acts.shape)
        acts = acts * attn_mask[:, :, None]
        tracer.log('output (act*mask):', acts.shape)
        acts = acts.sum(1) / attn_mask.sum(1)[:, None] # (8, 512)
        tracer.log('output (averaged across batch):', acts.shape) # to feed into probe
        acts.save()
        # logits = probe(acts)
        # tracer.log(logits.shape)
        # metric = []
        # for i in range(len(labels)): 
        #     if labels[i] == 0: 
        #         metric.append(probe(acts))
        #         # metric.append(probe(acts)),
        #     else: 
        #         labels[i] = - probe(acts)
        # tracer.log(labels)
        # probe(acts)
        metric = t.where(
                t.tensor(labels) == 0, 
                probe(acts), 
                -probe(acts)
        )
        metric.save()
        
labels
metric

output : torch.Size([8, 126, 512])
output (act*mask): torch.Size([8, 126, 512])
output (averaged across batch): torch.Size([8, 512])


tensor([-4.7353, 10.0931,  8.0942,  5.1765, 13.8580, 11.6943,  7.6786,  5.5229],
       grad_fn=<WhereBackward0>)

#### debug 3: 
- TODO: seem to be stuck now at the activation patching approx code
- RuntimeError: The size of tensor a (142) must match the size of tensor b (40) at non-singleton dimension 0


In [ ]:
data = get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED)
len(data) 

#### activation


In [ ]:

# Dictionary to store activations
activations = {}
# Hook function
def hook_fn(module, input, output):
    activations[model.gpt_neox.layers[layer]] = output

# Register hook
handle = model.gpt_neox.layers[layer].register_forward_hook(hook_fn)

# Forward pass
output = model(input_data)

# Access the activations
print("Activations from conv1:", activations["conv1"])

#### nn aside: custom function

In [ ]:
import nnsight

# Take a tensor and return the sum of its elements

def tensor_sum(tensor):
    flat = tensor.flatten()
    total = 0
    for element in flat:
        total += element.item()
    return t.tensor(total)

with model.trace(input) as tracer:
    # Specify the function name and its arguments (in a coma-separated form) to add to the intervention graph
    custom_sum = nnsight.apply(tensor_sum, tiny_model.layer1.output).save()
    sum = tiny_model.layer1.output.sum()
    sum.save()
    
print(custom_sum, sum)

In [44]:
import nnsight

# Take a tensor and return the sum of its elements

def tensor_sum(tensor):
    flat = tensor.flatten()
    total = 0
    for element in flat:
        total += element.item()
    return t.tensor(total)

with tiny_model.trace(input) as tracer:

    # Specify the function name and its arguments (in a coma-separated form) to add to the intervention graph
    custom_sum = nnsight.apply(tensor_sum, tiny_model.layer1.output).save()
    sum = tiny_model.layer1.output.sum()
    sum.save()
    
print(custom_sum, sum)

tensor(0.6482) tensor(0.6482)


In [24]:
# find most influential features
# based on linear approx of the casual score for each SAE latent 
# given latents, model, clean
from attribution import patching_effect
# n_batches = 25
n_batches = 2
batch_size = 4

running_total = 0
nodes = None

# for batch_idx, (clean, labels, _) in tqdm(enumerate(get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED)), total=n_batches):
#     if batch_idx == n_batches:
#         break
#     print(batch_idx, clean, labels)

### goal: 
### estimate causal impact based on integrated gradients 
### given text/data (input) 
### model, dictionaries (model components)
### metric function, labels (measure of causal impact)
### via attribution patching as approximation (ig: more accurate)
clean = data[0][0]
labels = data[0][1]

effects, _, _, _ = patching_effect(
    clean, # input
    None, # patch
    model,
    submodules, # index 
    dictionaries, # components
    metric_fn,
    metric_kwargs=dict(labels=labels),
    method='ig'
)

You're using a GPTNeoXTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


AttributeError: Above exception when execution Node: 'fetch_attr_1' in Graph: '140028619712016'

In [25]:
type(submodules[1])

nnsight.envoy.Envoy

In [23]:
submodules[1]

GPTNeoXSdpaAttention(
  (rotary_emb): GPTNeoXRotaryEmbedding()
  (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
  (dense): Linear(in_features=512, out_features=512, bias=True)
  (attention_dropout): Dropout(p=0.0, inplace=False)
)

In [34]:
submodules # (5*3 + embed)
# (layer 0, 1, 2, 3, 4 + embed)

GPTNeoXSdpaAttention(
  (rotary_emb): GPTNeoXRotaryEmbedding()
  (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
  (dense): Linear(in_features=512, out_features=512, bias=True)
  (attention_dropout): Dropout(p=0.0, inplace=False)
)

In [45]:
# submodules: [embedding, attention(rotary), mlp, layer]
is_tuple = {}
count = 0
with model.trace("_") as tracer:

    for submodule in submodules:
        # embedding
        if type(submodule.output) == tuple: 
            tracer.log(submodule.output)
        # is_tuple[submodule] = type(submodule.output) == tuple
        # tracer.log(submodule)
        # tracer.log(submodule.output.shape) 
        # tracer.log(type(submodule.output))
        count += 1
        tracer.log(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [28]:
labels

[1, 1, 1, 1, 1, 1, 1, 1]

In [53]:
def metric_fn(model, labels=None):
    attn_mask = model.inputs[1]['attention_mask']
    acts = model.gpt_neox.layers[layer].output[0]
    acts = acts * attn_mask[:, :, None]
    acts = acts.sum(1) / attn_mask.sum(1)[:, None]
    print('here')
    
    return t.where(
        labels == 0,
        probe(acts),
        - probe(acts)
    )

In [57]:
from activation_utils import SparseAct
# named tuple: Creates a new tuple subclass with named fields for each item
from collections import namedtuple
EffectOut = namedtuple('EffectOut', ['effects', 'deltas', 'grads', 'total_effect'])

# DEBUGGING = False

# if DEBUGGING:
#     tracer_kwargs = {'validate' : True, 'scan' : True}
# else:
#     tracer_kwargs = {'validate' : False, 'scan' : False}

def _pe_ig(
        clean,
        patch,
        model,
        submodules,
        dictionaries,
        metric_fn,
        steps=10,
        metric_kwargs=dict(),
):
    
    # first run through a test input to figure out which hidden states are tuples
    is_tuple = {}
    with model.trace("_") as tracer:

        for submodule in submodules:
            # tracer.log(submodule)
            is_tuple[submodule] = type(submodule.output) == tuple

    hidden_states_clean = {}
    # to gather hidden states of a batch of inputs (8, embd)
    
    with model.trace(clean, **tracer_kwargs) as tracer, t.no_grad():
        # step through components of model, one by one
        # get dict/sae for the component
        # save features (32768); feature node activation
        # save residual (error term); error node activation
        for submodule in submodules:
            dictionary = dictionaries[submodule]
            x = submodule.output
            tracer.log(submodule.output)
            tracer.log(submodule.output.shape)
            if is_tuple[submodule]:
                x = x[0]
            f = dictionary.encode(x) #32768
            tracer.log(f.shape)
            x_hat = dictionary.decode(f) #512
            residual = x - x_hat
            tracer.log(residual.shape)
            hidden_states_clean[submodule] = SparseAct(act=f.save(), res=residual.save())
        tracer.log('submodules done')
        ## just getting classification logits 
        metric_clean = metric_fn(model, **metric_kwargs).save()
        tracer.log('metric', metric_clean.shape)
    # {submodule : hidden state (saved as SparseAct class)}
    hidden_states_clean = {k : v.value for k, v in hidden_states_clean.items()}

    if patch is None:
        ## zeros
        hidden_states_patch = {
            k : SparseAct(act=t.zeros_like(v.act), res=t.zeros_like(v.res)) for k, v in hidden_states_clean.items()
        }
        total_effect = None
    else:
        ## hidden state of 'corrupted'; patched features
        hidden_states_patch = {}
        with model.trace(patch, **tracer_kwargs), t.no_grad():
            for submodule in submodules:
                dictionary = dictionaries[submodule]
                x = submodule.output
                if is_tuple[submodule]:
                    x = x[0]
                f = dictionary.encode(x)
                x_hat = dictionary.decode(f)
                residual = x - x_hat
                hidden_states_patch[submodule] = SparseAct(act=f.save(), res=residual.save())
            ## 
            metric_patch = metric_fn(model, **metric_kwargs).save()
        total_effect = (metric_patch.value - metric_clean.value).detach()
        hidden_states_patch = {k : v.value for k, v in hidden_states_patch.items()}

    effects = {}
    deltas = {}
    grads = {}
    for submodule in submodules:
        dictionary = dictionaries[submodule]
        clean_state = hidden_states_clean[submodule]
        patch_state = hidden_states_patch[submodule]
        with model.trace(**tracer_kwargs) as tracer:
            metrics = []
            fs = []
            ## linear approx
            for step in range(steps):
                alpha = step / steps
                f = (1 - alpha) * clean_state + alpha * patch_state
                f.act.retain_grad()
                f.res.retain_grad()
                fs.append(f)
                with tracer.invoke(clean, scan=tracer_kwargs['scan']):
                    if is_tuple[submodule]:
                        submodule.output[0][:] = dictionary.decode(f.act) + f.res
                    else:
                        submodule.output = dictionary.decode(f.act) + f.res
                    metrics.append(metric_fn(model, **metric_kwargs))
            metric = sum([m for m in metrics])
            metric.sum().backward(retain_graph=True) # TODO : why is this necessary? Probably shouldn't be, contact jaden

        mean_grad = sum([f.act.grad for f in fs]) / steps
        mean_residual_grad = sum([f.res.grad for f in fs]) / steps
        grad = SparseAct(act=mean_grad, res=mean_residual_grad)
        delta = (patch_state - clean_state).detach() if patch_state is not None else -clean_state.detach()
        effect = grad @ delta

        effects[submodule] = effect
        deltas[submodule] = delta
        grads[submodule] = grad

    return EffectOut(effects, deltas, grads, total_effect)

["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
 'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and L

In [55]:
DEBUGGING = True

if DEBUGGING:
    tracer_kwargs = {'validate' : True, 'scan' : True}
else:
    tracer_kwargs = {'validate' : False, 'scan' : False}

_pe_ig(
        clean,
        None,
        model,
        submodules,
        dictionaries,
        metric_fn,
        steps=10,
        metric_kwargs=dict(labels=labels),
)
# effects, _, _, _ = patching_effect(
#     clean, # input
#     None, # patch
#     model,
#     submodules, # index 
#     dictionaries, # components
#     metric_fn,
#     metric_kwargs=dict(labels=labels),
#     method='ig'
# )

FakeTensor(..., size=(8, 126, 512), grad_fn=<EmbeddingBackward0>)
torch.Size([8, 126, 512])
torch.Size([8, 126, 32768])
torch.Size([8, 126, 512])
(FakeTensor(..., size=(8, 126, 512), grad_fn=<ViewBackward0>), DynamicCache(), None)
(torch.Size([8, 126, 512]), DynamicCache(), None)


TypeError: unsupported operand type(s) for -: 'tuple' and 'Parameter'

In [10]:
len(submodules), len(dictionaries)

(16, 16)

In [11]:
submodules

[Embedding(50304, 512),
 GPTNeoXSdpaAttention(
   (rotary_emb): GPTNeoXRotaryEmbedding()
   (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
   (dense): Linear(in_features=512, out_features=512, bias=True)
   (attention_dropout): Dropout(p=0.0, inplace=False)
 ),
 GPTNeoXMLP(
   (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
   (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
   (act): GELUActivation()
 ),
 GPTNeoXLayer(
   (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
   (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
   (post_attention_dropout): Dropout(p=0.0, inplace=False)
   (post_mlp_dropout): Dropout(p=0.0, inplace=False)
   (attention): GPTNeoXSdpaAttention(
     (rotary_emb): GPTNeoXRotaryEmbedding()
     (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
     (dense): Linear(in_features=512, out_features=512, bias=True)
     

In [ ]:
### patching effect by integrated gradients 

###

## 4. edit / inform ReFT based on the graph? 
- what do we mean by edit? 
- what do we mean by finetune? (reduce a direction)

- we need to define it (what have others done, what is the challenge)